# Scraping Maryland Business Licenses with Selenium

Maryland has a [great portal](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp) for searching business licenses, but the only problem is you have to check a box in order to get in.

1. Try to visit [the public search page](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp)
2. Get redirected to a "I agree to this" page. Click that you've read the disclaimer, click Enter the Site.
3. Click "Search License Records" down at the bottom of the page
4. You're now on the search page! From the "Jurisdiction" dropdown, select "Statewide"
5. In the "Trade Name" field, type "Vap%" to try to find vape shops
6. Click "Next" in the bottom right-hand corner to go to the next page
7. Click "Click for detail" to see the details for a specific business license.

That's a lot of stuff! **Let's get to work.**

## Preparation

### When you search for a business license, what URL should Selenium try to visit first?

In [54]:
from selenium import webdriver
driver = webdriver.Chrome()

In [55]:
driver.get("https://jportal.mdcourts.gov/license/pbPublicSearch.jsp")

**It isn't going to work, though! It's going to redirect to that intro page.** You can use *Incognito mode* to go back through the "Check the box, etc" series of pages.

### How will you identify the checkbox to check it?

Selenium can find elements by:

- name
- Class
- ID
- CSS selector (**ASK ME WHAT THIS IS** if you don't know)
- XPath (**ASK ME WHAT THIS IS** because you definitely don't know)
- Link text
- Partial link text

So in other words, what's unique about this element?

- *TIP: I have a secret awesome way to do this, but you have to ask.*

In [56]:
checkbox = driver.find_element_by_xpath('//*[@id="checkbox"]')
checkbox.click()

### How will you identify the button to select it, or the form to submit it?

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

In [57]:
submit = driver.find_element_by_xpath('/html/body/table/tbody/tr[7]/td/form/div/input[2]')
submit.click()

### Once you're on the next page, how will you click the "SEARCH LICENSE RECORDS" link or follow its URL?

In [58]:
search_license_records = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[6]/td[2]/a[2]')
search_license_records.click()

### Now you're on the form page. How do you pick "Statewide" for the jurisdiction dropdown?

In [59]:
from selenium.webdriver.support.ui import Select

In [60]:
select_tag = driver.find_element_by_name('slcJurisdiction')
select = Select(select_tag)
select.select_by_visible_text('Statewide')

### How do you type "vap%" into the Trade Name field?

In [61]:
from selenium.webdriver.common.keys import Keys

In [62]:
trade_name = driver.find_element_by_xpath('//*[@id="txtTradeName"]')
trade_name.send_keys('vap%')

### How do you click the submit button or submit the form?

In [63]:
trade_name.send_keys(Keys.RETURN)

### How can you find and click the 'Next' button on the search results page?

In [11]:
Next_button = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr/td[3]/a/nobr')
Next_button.click()

# Okay, let's scrape!

### Use Selenium to search for vape shops statewide, and then click through until it reaches the last page.

Don't scrape yet!

In [12]:
while True:
    try:
        driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr/td[3]/a/nobr').click()
    except:
        break

### Use Selenium to scrape the first page of search results for vape shops statewide (well, we'll try).

- *TIP: You can find elements/text using Selenium, or use BeautifulSoup with `doc = BeautifulSoup(driver.page_source)`*

I've included some BeautifulSoup code that might be helpful. If you use it, **ask me how I made it.** It's important.

In [64]:
import requests
from bs4 import BeautifulSoup
doc = BeautifulSoup(driver.page_source, 'html.parser')

In [65]:
# IF YOU ARE USING BEAUTIFULSOUP, HERE IS SOME SAMPLE CODE
# IF YOU USE THIS CODE, ASK ME HOW I MADE IT. IT'S IMPORTANT.
business_headers = doc.find_all('tr',class_='searchfieldtitle')
len(business_headers)

5

In [66]:
# You'll probably need to find specific tds inside
# of the rows instead of looking at the whole thing.
for header in business_headers:
    rows = header.find_next_siblings('tr')
    print("HEADER is", header.text.strip())
    print("ROW 0 IS", rows[0].text.strip())
    print("ROW 1 IS", rows[1].text.strip())
    print("ROW 2 IS", rows[2].text.strip())
    print("ROW 3 IS", rows[3].text.strip())
    print("----")

HEADER is 1.
VAPE IT STORE II
ROW 0 IS AMIN NARGIS
Lic. Status: Issued
ROW 1 IS 1015 S SALISBURY BLVD
License: 22173808
ROW 2 IS SALISBURY, MD 21801
Issued Date: 4/27/2017
ROW 3 IS Wicomico County
----
HEADER is 2.
VAPE IT STORE I
ROW 0 IS AMIN NARGIS
Lic. Status: Issued
ROW 1 IS 1724 N SALISBURY BLVD UNIT 2
License: 22173807
ROW 2 IS SALISBURY, MD 21801
Issued Date: 4/27/2017
ROW 3 IS Wicomico County
----
HEADER is 3.
VAPEPAD THE
ROW 0 IS ANJ DISTRIBUTIONS LLC
Lic. Status: Issued
ROW 1 IS 2299 JOHNS HOPKINS ROAD
License: 02104436
ROW 2 IS GAMBRILLS, MD 21054
Issued Date: 4/05/2017
ROW 3 IS Anne Arundel County
----
HEADER is 4.
VAPE FROG
ROW 0 IS COX TRADING COMPANY L L C
Lic. Status: Issued
ROW 1 IS 110 S. PINEY RD
License: 17165957
ROW 2 IS CHESTER, MD 21619
Issued Date: 5/31/2017
ROW 3 IS Queen Anne's County
----
HEADER is 5.
VAPE FROG
Pending *
ROW 0 IS COX TRADING LLC
Lic. Status: Pending
ROW 1 IS 346 RITCHIE HIGHWAY
ROW 2 IS SEVERNA PARK, MD 21146
ROW 3 IS Anne Arundel County
---

In [67]:
vape_results = []
business_headers = doc.find_all('tr',class_='searchfieldtitle')
for header in business_headers:
    rows = header.find_next_siblings('tr')
    header1 = header.find_all('td')[1].text.strip()
    name = rows[0].find_all('td')[1].text.strip()
    address = rows[1].find_all('td')[1].text.strip()
    post = rows[2].find_all('td')[1].text.strip()
    county = rows[3].text.strip()
 
    
    HEADER = header.find_all('td')[2].text.strip()
    Lic_Status =  rows[0].find_all('td')[2].text.strip()
    License = rows[1].find_all('td')[2].text.strip()
    Issued_Date = rows[2].find_all('td')[2].text.strip()
    
    Lic_Status = Lic_Status.split(':')
    if len(Lic_Status) > 1:
        Lic_Status = Lic_Status[1].strip()
   
    License = License.split(':')
    if len(License) > 1:
        License = License[1].strip()
    
    Issued_Date = Issued_Date.split(':')
    if len(Issued_Date) > 1:
        Issued_Date = Issued_Date[1].strip()
        
    vape_results.append({
        'HEADER': header,
        'name': name,
        'address': address,
        'post': post,
        'county': county,
        'HEADER': HEADER,
        'Lic. Status': Lic_Status,
        'License': License,
        'Issued Date': Issued_Date
})
print(vape_results)

[{'License': '22173808', 'HEADER': '', 'address': '1015 S SALISBURY BLVD', 'Issued Date': '4/27/2017', 'post': 'SALISBURY, MD 21801', 'Lic. Status': 'Issued', 'county': 'Wicomico County', 'name': 'AMIN NARGIS'}, {'License': '22173807', 'HEADER': '', 'address': '1724 N SALISBURY BLVD UNIT 2', 'Issued Date': '4/27/2017', 'post': 'SALISBURY, MD 21801', 'Lic. Status': 'Issued', 'county': 'Wicomico County', 'name': 'AMIN NARGIS'}, {'License': '02104436', 'HEADER': '', 'address': '2299 JOHNS HOPKINS ROAD', 'Issued Date': '4/05/2017', 'post': 'GAMBRILLS, MD 21054', 'Lic. Status': 'Issued', 'county': 'Anne Arundel County', 'name': 'ANJ DISTRIBUTIONS LLC'}, {'License': '17165957', 'HEADER': '', 'address': '110 S. PINEY RD', 'Issued Date': '5/31/2017', 'post': 'CHESTER, MD 21619', 'Lic. Status': 'Issued', 'county': "Queen Anne's County", 'name': 'COX TRADING COMPANY L L C'}, {'License': [''], 'HEADER': 'Pending *', 'address': '346 RITCHIE HIGHWAY', 'Issued Date': [''], 'post': 'SEVERNA PARK, MD 

### Save these into `vape-results.csv`

In [68]:
import pandas as pd

In [69]:
df = pd.DataFrame(vape_results)
df.to_csv('vape_results.csv', index=False)

### Open `vape-results.csv` to make sure there aren't any extra weird columns

In [70]:
pd.read_csv('vape_results.csv')

,HEADER,Issued Date,Lic. Status,License,address,county,name,post
0,NaN,4/27/2017,Issued,22173808,1015 S SALISBURY BLVD,Wicomico County,AMIN NARGIS,"SALISBURY, MD 21801"
1,NaN,4/27/2017,Issued,22173807,1724 N SALISBURY BLVD UNIT 2,Wicomico County,AMIN NARGIS,"SALISBURY, MD 21801"
2,NaN,4/05/2017,Issued,02104436,2299 JOHNS HOPKINS ROAD,Anne Arundel County,ANJ DISTRIBUTIONS LLC,"GAMBRILLS, MD 21054"
3,NaN,5/31/2017,Issued,17165957,110 S. PINEY RD,Queen Anne's County,COX TRADING COMPANY L L C,"CHESTER, MD 21619"
4,Pending *,[''],Pending,[''],346 RITCHIE HIGHWAY,Anne Arundel County,COX TRADING LLC,"SEVERNA PARK, MD 21146"


## Use Selenium to scrape ALL pages of results, save the results into `vape-results-all.csv`.

In [ ]:
while True:
    doc = BeautifulSoup(driver.page_source, 'html.parser')
    vape_results = []
    business_headers = doc.find_all('tr',class_='searchfieldtitle')
    for header in business_headers:
        rows = header.find_next_siblings('tr')
        header1 = header.find_all('td')[1].text.strip()
        name = rows[0].find_all('td')[1].text.strip()
        address = rows[1].find_all('td')[1].text.strip()
        post = rows[2].find_all('td')[1].text.strip()
        county = rows[3].text.strip()
 
        HEADER = header.find_all('td')[2].text.strip()
        Lic_Status =  rows[0].find_all('td')[2].text.strip()
        License = rows[1].find_all('td')[2].text.strip()
        Issued_Date = rows[2].find_all('td')[2].text.strip()
    
        Lic_Status = Lic_Status.split(':')
        if len(Lic_Status) > 1:
            Lic_Status = Lic_Status[1].strip()
   
        License = License.split(':')
        if len(License) > 1:
            License = License[1].strip()
    
        Issued_Date = Issued_Date.split(':')
        if len(Issued_Date) > 1:
            Issued_Date = Issued_Date[1].strip()
        
        vape_results.append({
            'HEADER': header,
            'name': name,
            'address': address,
            'post': post,
            'county': county,
            'HEADER': HEADER,
            'Lic. Status': Lic_Status,
            'License': License,
            'Issued Date': Issued_Date
    })
    print(vape_results)
    try:
        driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr/td[3]/a/nobr').click()
    except:
        break

In [ ]:
df = pd.DataFrame(licenses)
df.to_csv('vape_all.csv', index=False)
pd.read_csv('vape_all.csv')